# 1. Validating restructured data against a schema using a spreadsheet

<div class="alert alert-block alert-warning">
    <b>Learning outcomes:</b>
    <br>
    <ul>
        <li>Understand, and employ, standard definitions to write a JSON schema for data validation.</li>
        <li>Perform data validation using Microsoft Excel.</li>
        <li>Learn how to validate machine-readable data in online applications against a defined schema.</li>
    </ul>
</div>

---

## 1.1 Creating a JSON schema

When you produced your machine-readable file in Lesson 1, you came up with your own approach to how to structure the header row and the data. You named the columns yourself, and decided on how many, and what data should be in them. You did so by reviewing the source data.

However, there are problems with that approach:

- How will you maintain consistency between updates? 
- How will different people, working on similar data or consecutive updates, ensure consistency between themselves?
- How will you know if the restructured file you've created is actually machine-readable?

If a programmer wanted to build an application - say an app that helps people find the nearest bus-stop - and you change update structure from release to release, they will not be able to automate their program, and that may prevent them from building it at all.

The first step in the data lifecycle isn't simply heading into your data and restructuring it. First you need to define a `schema`.

![Data lifecycle](images/data-lifecycle-en.jpg)

Your `schema` defines the structural metadata for your wrangling process. This is not the format your input data arrive in, but it is what you require it to look like when you're done.

The standard terms used are defined by [JSON Schema](https://json-schema.org/learn/) and further refined for use in structuring CSV data in Frictionless Data's [Table Schema](https://specs.frictionlessdata.io/table-schema/).

In simple terms, you need to specify the columns which an input CSV or Excel-file will be restructured into. The new columns are defined by the fields in your schema. These target fields are likely to be those in your database, or in your analytical software. Until your input data conform to this structure, your data will not validate.

### 1.1.1 Minimum valid requirements

A minimum valid schema requires a `name` to identify the schema, and a single, minimally-valid `field` containing a `name` and `type`:

    {
        "name": "A simple name",
        "fields": [
            {
                "name": "Field name, e.g. 'column_name'",
                "type": "Valid data type, e.g. 'string', 'number'"
                }
            }
        ]
    }

Everything else is optional, unless specifically required by that field-type.

A JSON schema looks similar to a Python dictionary, but be very careful with `,` line-endings. The last item in a list must NOT have a comma. 

> You can think of a schema `field` as a `column` in a table, or a `field` in a database. Fields have a `type`, such as integer or text.

The `fields` value is a list, or - in JSON terminology - an `array` of dictionary `objects`. Each field, unsurprisingly, has a `name`, `title` and `description`, of which only the `name` is required. 

### 1.1.2 Types

Fields also have a `type`. This describes the data expected and limits the actions which can be performed during the wrangling process:

* `string`: any text-based string.
* `number`: any number-based value, including integers and floats.
* `integer`: any integer-based value.
* `boolean`: a boolean [`true`, `false`] value.
* `object`: any valid JSON data.
* `array`: any valid array-based data.
* `date`: any date without a time. Must be in ISO8601 format, `YYYY-MM-DD`.
* `datetime`: any date with a time. Must be in ISO8601 format, with UTC time specified (optionally) as `YYYY-MM-DD hh:mm:ss Zz`.
* `year`: any year, formatted as `YYYY`.

There are more [types and formats](https://specs.frictionlessdata.io/table-schema/#types-and-formats) like `geojson`, `geopoints` and variations on dates.

### 1.1.3 Constraints

In addition, these data can be `constrained`:

* `required`: boolean, indicates whether this field is compulsory (but blank values in the input column are permitted and will be set to the `missing` default)
* `unique`: boolean, if `true` then all values for that input column must be unique
* `minimum`: `integer` / `number`, as appropriate defining min number of characters in a string, or the min values of numbers or integers
* `maximum`: `integer` / `number`, as appropriate defining max number of characters in a string, or the max values of numbers or integers
* `enum`: a list (or array) of unique terms permitted in this field

As an example:

    {
        "name": "A simple name",
        "fields": [
            {
                "name": "Field name, e.g. 'column_name'",
                "type": "Valid data type, e.g. 'string', 'number'",
                "constraints": {
                    "enum": ["cheddar", "gouda", "other"]
                }
            }
        ]
    }

Again, there are other [constraints](https://specs.frictionlessdata.io/table-schema/#constraints), such as `pattern`, `maxLength`, `minLength` you can use as well.

### 1.1.4 Other properties

There are also special properties you can add to your schema that are not part of the `fields` definitions:

* `missingValues`: defines which terms in your data should be treated as missing values, e.g. `-`, `NaN`, `..`, etc. This must be presented as a list, with terms defined as strings, e.g. `["NaN", ".."]`

### 1.1.5 Example schema

As an example, let's imagine we want our destination data to conform to the following structure:

    =========  ============  =============  ========  ================  =====================  =============  ========================
      la_code        ba_ref  occupant_name  postcode  occupation_state  occupation_state_date  prop_ba_rates  occupation_state_reliefs
    =========  ============  =============  ========  ================  =====================  =============  ========================
    E06000044  177500080710  A company       PO5 2SE              True             2019-04-01          98530  [small_business, retail]
    =========  ============  =============  ========  ================  =====================  =============  ========================

The complete schema for this example is then:

In [ ]:
{
    "name": "test_schema",
    "title": "Test Schema",
    "description": "A test Schema",
    "fields": [
        {
            "name": "la_code",
            "title": "Local authority code",
            "type": "string",
            "description": "Standard code for local authority."
        },
        {
            "name": "ba_ref",
            "title": "Billing reference",
            "type": "string",
            "description": "Unique code for a specific hereditament. May be multiple rows for history."
        },
        {
            "name": "prop_ba_rates",
            "title": "Property billing rates",
            "type": "number",
            "description": "Actual rates paid by a specific ratepayer."
        },
        {
            "name": "occupant_name",
            "title": "Occupier name",
            "type": "string",
            "description": "Name of the ratepayer."
        },
        {
            "name": "postcode",
            "title": "Postcode",
            "type": "string",
            "description": "Full address or postcode of ratepayer."
        },
        {
            "name": "occupation_state",
            "title": "Occupation state",
            "type": "boolean",
            "description": "Occupation status, void or occupied."
        },
        {
            "name": "occupation_state_date",
            "title": "Date of occupation state",
            "type": "date",
            "description": "Date of the start of status in occupation_state."
        },
        {
            "name": "occupation_state_reliefs",
            "title": "Occupation state reliefs",
            "type": "array",
            "description": "Array of the categories of reliefs / exemptions applied.",
            "constraints": {
                "enum": [
                    "small_business", 
                    "rural", 
                    "charity", 
                    "enterprise_zone", 
                    "vacancy", 
                    "hardship",
                    "retail", 
                    "discretionary", 
                    "exempt", 
                    "transitional", 
                    "other"
                ]
            }
        }
    ],
    "missingValues": ["NaN", ".."]
}

__Save__ your JSON schema in any ordinary text editor (__NOT__ Word or anything that'll add XML to it) as a `.json` file (e.g. `my_schema.json`).

A really good text editor is [Notepad++](https://notepad-plus-plus.org/downloads/). Use this to create and save your JSON.

Now your turn:

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p>Using the restructured file you created in Lesson 1.1 create a JSON Schema. Remember that each <code>field</code> <code>name</code> must be the same as the column header name. Define the field <code>type</code> and - where you have them - any <code>missingValues</code>.</p>
</div>

In the next section, you'll learn how to specify the value types in a spreadsheet.

<div class="alert alert-block alert-info">
    <b>References:</b>
    <br>
    <ul>
        <li><a href="https://json-schema.org/learn/">JSON Schema</a></li>
        <li><a href="https://specs.frictionlessdata.io/table-schema/">Frictionless Data Table Schema</a></li>
        <li><a href="https://notepad-plus-plus.org/downloads/">Notepad++</a></li>
    </ul>
</div>

---

## 1.2 Apply data validation to cells in a spreadsheet

Your `types` - at this stage - are only a guide. You will have no feedback, or error messages like you get when running Python code, if any of the data types in your field columns are wrong. There are a few ways to get that feedback so you can correct things, but we'll start with data validation in spreadsheet cells.

The following is adapted from a [Microsoft Office tutorial](https://support.office.com/en-gb/article/apply-data-validation-to-cells-29fecbcc-d1b9-42c1-9d76-eff3ce5f7249). This approach will work in OpenOffice as well as Google Sheets, although the specific steps are different.

Microsoft has an example file you can [download](http://download.microsoft.com/download/9/6/8/968A9140-2E13-4FDC-B62C-C1D98D2B0FE6/Data%20Validation%20Examples.xlsx).

### 1.2.1 Specify validation for data types

The process is straightforward:

1. Select the cells in a specific column you wish to limit by type
2. Select __Data > Data Tools > Data Validation__.

  ![Excel data validation](images/excel-data-validation.png)

3. On the __Settings__ tab, under __Allow__, select one of:

  ![Validation settings](images/excel-validation-settings.jpg)
 
  - __Whole Number__: restrict the cell to accept only `integer` values.
  - __Decimal__: restrict the cell to accept only `float` or `number` values.
  - __List__: pick data from a drop-down list, and limited by values constrained by `enum`.
  - __Date__: restrict the cell to accept only `date`.
  - __Time__: restrict the cell to accept only `datetime`.
  - __Text Length__: restrict the length of the text, equivalent to constraints `maxLength`.
  - __Custom__: for custom formula.
 
4. Under __Data__, you can select a condition:

  - between
  - not between
  - equal to
  - not equal to
  - greater than
  - less than
  - greater than or equal to
  - less than or equal to

5. Set the other required values, based on what you chose for __Allow__ and __Data__. For example, if you select between, then select the __Minimum:__ and __Maximum:__ values for the cell(s).
6. Select the __Ignore blank__ checkbox if you want to ignore blank spaces (i.e. for missing data). Note, though, that Excel doesn't have any room for special characters you may be using as a marker for missing data (e.g. `..`) so these would be raised as errors.
7. Select __OK__.

Now - only for new data - if a user tries to enter a value that is not valid, a pop-up appears with the message, "This value doesn’t match the data validation restrictions for this cell." We'll run validation on your existing data shortly, but first a detour into `lists`.

### 1.2.2 Lists are a special type

Before you can validate a `list` type, you need to specify valid terms. In Excel, this requires an [extra set of steps](https://support.office.com/en-us/article/create-a-drop-down-list-7693307a-59ef-400a-b769-c5402dce407b).

1. Create a new worksheet in Excel, and there list the terms you want to set as valid values. You can quickly convert your list to a table by selecting any cell in the range, and pressing __Ctrl+T__ (this may differ from version to version in Excel, or entirely in Open Office or other spreadsheet applications).

  ![List terms](images/excel-list-terms.png)
  
2. Add your list data and format it as a Table (__Home tab > Styles > Format as Table__).
3. You can name your table from the Table tools tab - this one could be named "CityTable".  This will help you keep track of multiple tables.
4. In the validation process listed above, go to 3. and select __List__, then add a named range or table name for your list. 
5. Specify a source for your terms via __Data tab > Data Validation > Allow List > Source__. Then specify your list of terms as any of:

  ![List source](images/excel-list-source.png)

  - You can select the list sheet and range directly (e.g. `=Sheet1!A4:A10`)
  - Convert your list to a table with __Ctrl+T__, then from the __Table Design__ tab give your table a name, permitting you to reference the table name and column (e.g. `=CityTable[City]`)
  - From the __Formulas__ tab select __Name Manager__, create a __New__ item with an appropriate name (e.g. `CityList`), and reference the cells (e.g. `=Sheet1!A4:A10`), which then lets you reference your list anywhere (e.g. `=CityList`)

### 1.2.3 Validate and get error messages for your existing data

After you've specified validation rules on your existing data you might be disappoined. Excel does not automatically notify you whether these cells contain invalid data. Here's a quick way to [highlight existing invalid cells](https://support.office.com/en-us/article/more-on-data-validation-f38dee73-9900-4ca6-9301-8a5f6e1f0c4c) by circling the values:

  ![Circle invalid data](images/excel-circled-cell.gif)

1. To apply the circles, select the cells you want to evaluate and go to __Data > Data Tools > Data Validation > Circle Invalid Data__.

  ![List terms](images/excel-data-circle.png)
  
2. If you correct an invalid entry, the circle disappears automatically.
3. To remove data validation for a cell, select it, and then go to __Data > Data Tools > Data Validation > Settings > Clear All__.

Now your turn:

<div class="alert alert-block alert-success">
    <p><b>Exercise:</b></p>
    <p>Using the restructured file you created in Lesson 1.1 specify validation criteria for each column based on the JSON Schema you created. Check for invalid data and correct where necessary.</p>
    <p>One thing you might notice in your data ... sometimes you have an invalid integer specified as a range, e.g. <code>200-210</code>. Here are some ideas about how to deal with that.</p>
    <ul>
        <li><b>Ranges instead of numbers</b>: if, e.g. your range is <code>200-210</code>, you could reset this value as <code>200</code>, or <code>210</code> or even the range average <code>205</code>. Whatever you decide, document your decision in your metadata file.</li>
        <li><b>Date ranges</b>: the same goes for dates, although you should be careful ... the likelihood is that a value applies from the end of the date range, not the beginning or middle, so e.g. <code>2008-2009</code> is most likely to be <code>2009</code></li>
    </ul>
</div>

<div class="alert alert-block alert-info">
    <b>References:</b>
    <br>
    <ul>
        <li><a href="https://support.office.com/en-gb/article/apply-data-validation-to-cells-29fecbcc-d1b9-42c1-9d76-eff3ce5f7249">Apply data validation to cells</a></li>
        <li><a href="https://support.office.com/en-us/article/create-a-drop-down-list-7693307a-59ef-400a-b769-c5402dce407b">Create a drop-down list</a></li>
        <li><a href="https://support.office.com/en-us/article/more-on-data-validation-f38dee73-9900-4ca6-9301-8a5f6e1f0c4c">More on data validation</a></li>
    </ul>
</div>

---

## 1.3 Saving your validated file as a comma-separated-value

Comma separated value files (`.csv`) are text files in which the comma character `,` separates each field of text. Where a comma appears in the value - whether a `string` or `number` - the value is then surrounded by quotation marks, e.g. `100, 200, "20,000"` indicates three values in three separate fields.

You can change the separator character that is used in both delimited and .csv text files, and there are a wide range of formats (e.g. `;`, `*`). There are any number of reasons for this, and it is part of the reason that CSV-formatted files are not the cure-all we would hope for ensuring consistency in open data.

In Excel, you can [export a spreadsheet](https://support.office.com/en-gb/article/import-or-export-text-txt-or-csv-files-5250ac4c-663c-47ce-937b-339e391393ba) as a CSV using __Save As__ as follows:

1. Go to __File > Save As__.
2. Click __Browse__.
3. In the __Save As__ dialog box, under __Save as type__ box, choose the text file format for the worksheet; for example, click __CSV (Comma delimited)__.
4. Browse to the location where you want to save the new text file, and then click __Save__.

> You are only able to export the current worksheet (i.e. the one in view when you complete this process) to the new CSV file. You can save other worksheets as separate text files by repeating this procedure for each worksheet.

> All spreadsheet-specific features will be lost. Formatting (bold, colours, etc), formulae and validation criteria will be removed leaving only the data in a text file.

---

## 1.4 Validating your data and JSON schema using CSVLint

In the next lesson, we'll learn how to validate your data using Python directly in a Jupyter Notebook, for now we'll use an online resource provided by the Open Data Institute called [CSVLint](https://csvlint.io/).

CSVLint provides the following guide on how to use it:

![CSVLint](images/csv-lint.jpg)

1. Enter the location of the CSV file you want to check, or upload it if it's not on the web already.
2. Tick __Add optional schema (in .json format)__ and enter its location or upload it.
3. Hit the big __Validate__ button.

You'll see a page that tells you how to improve your CSV file, if it needs improvement.

- __Errors__ are things you really need to fix, because they'll stop people from being able to use your data
- __Warnings__ are thing you should fix if you can because doing so will help people to use your data
- __Information__ messages are hints and tips of things that could make the data easier to use

CSVLint also lists a number of common errors for you to watch out for:

- __Invalid encoding__: if there are any odd characters in a file which could cause encoding errors
- __Line breaks__: if line breaks are not the same throughout the file
- __Undeclared header__: if you do not specify in a machine readable way whether or not your CSV has a header row
- __Ragged rows__: if every row in the file doesn't have the same number of columns
- __Blank rows__: if there are any blank rows
- __Stray/Unclosed quote__: if there are any unclosed quotes in the file
- __Whitespace__: if there is any whitespace between commas and double quotes around fields

<div class="alert alert-block alert-info">
    <b>References:</b>
    <br>
    <ul>
        <li><a href="https://csvlint.io/">CSVLint</a></li>
        <li><a href="https://datagraft.io/">DataGraft</a></li>
    </ul>
</div>

---

## 1.5 Lesson tutorial

<div class="alert alert-block alert-success">
    <p><b>Tutorial:</b></p>
    <p>Complete the processing of the file you started working with in Lesson 1.</p>
    <ul>
        <li><b>JSON schema</b>: Define a JSON schema to desribe and specify validation criteria for your dataset.</li>
        <li><b>Check for invalid data</b>: Specify validation criteria in your spreadsheet application, check for invalid data and correct where necessary..</li>
        <li><b>Save as CSV</b>: Export your machine-readable data from your spreadsheet application and save it as a <code>.csv</code>.</li>
        <li><b>Validate</b>: Upload your CSV and JSON schema onto <a href="https://csvlint.io/">CSVLint</a> and ensure you are successfully validated. Fix any errors you receive.</li>
    </ul>
</div>

Please complete the tutorial before continuing with this series.